<a href="https://colab.research.google.com/github/johnchoi44/ProjectTwitter/blob/main/Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# connect to Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
cd drive/MyDrive/Lawyers/Gemini/

/content/drive/MyDrive/Lawyers/Gemini


In [15]:
pip install -q -U google-genai

In [22]:
# import dependencies

from google import genai
import pandas as pd
import json
import ast
import re
from openpyxl import load_workbook

from datetime import datetime
from google.api_core.exceptions import ClientError

In [17]:
# input your API Key for gemini

client = genai.Client(api_key="Your Key")

In [18]:
# first move the excel file to the path

com = "Cooley" # change this
# open excel file
path = "/content/drive/MyDrive/Lawyers/Gemini/{}/{}.xlsx".format(com, com)
df = pd.read_excel(path)
try:
  lawfirm = df["Lawfirm"].tolist()
except:
  pass

In [19]:
edu = df['Education']

In [20]:
prompt = "Convert the following education history into a Python dictionary with integer keys starting from 0. Each value should be a nested dictionary that includes fields such as school, year, degree, major (if available), and honors (if mentioned). Format the output as valid Python with no extra text or explanation."

In [24]:
# file paths
checkpoint_file = "edu_checkpoint.json"

# function to extract the dictionary block from LLM response
def extract_dict_from_markdown(response_text):
    match = re.search(r"```(?:python)?(.*?)```", response_text, re.DOTALL)
    if match:
        code_block = match.group(1).strip()
        # Remove any variable assignment like: education_history = {
        if '=' in code_block:
            code_block = code_block.split('=', 1)[1].strip()
        return ast.literal_eval(code_block)
    else:
        raise ValueError("No Python dictionary block found in the response.")

# load checkpoint if exists
def load_checkpoint():
  try:
    with open(checkpoint_file, 'r') as f:
      return json.load(f)
  except FileNotFoundError:
    return {"last_index": 0}

def save_checkpoint(index, error_msg=None):
  data = {"last_index": index}
  if error_msg:
    data["error_time"] = datetime.now().isoformat()
    data["error_msg"] = error_msg
  with open(checkpoint_file, 'w') as f:
    json.dump(data, f, indent=2)


In [25]:
wb = load_workbook(path)
ws = wb.active
last_col = ws.max_column + 1
# r = 2
first = True

In [30]:
checkpoint = load_checkpoint()
start_index = checkpoint["last_index"]

for idx in range(start_index, len(edu)):
  i = edu[idx]

  # skip if i is NaN or empty
  if not isinstance(i, str) or i.strip() == "" or str(i).lower() == "nan":
    print(f"Skipping empty entry at index {idx}")
    save_checkpoint(idx + 1)
    continue

  try:
    response = client.models.generate_content(
      model="gemini-2.0-flash",
      contents=str(i)+prompt
      )
    education_data = extract_dict_from_markdown(response.text)
    flattened = {}
    for key, value in education_data.items():
      for sub_key, sub_val in value.items():
        flattened[f"{key}_{sub_key}"] = sub_val

    if first:
      for j, col_name in enumerate(flattened.keys(), start=last_col):
        ws.cell(row=1, column=j, value=col_name)
      first = False

    for j, value in enumerate(flattened.values(), start=last_col):
      if isinstance(value,list):
        value = ", ".join(str(v) for v in value)
      ws.cell(row=idx + 2, column=j, value=value)

    # save workbook and checkpoint after each entry
    wb.save(path)
    save_checkpoint(idx + 1)

  except ClientError as e:
    if "RESOURCE_EXHAUSTED" in str(e):
      print(f"Quota error at index {idx}: {e}")
      save_checkpoint(idx, error_msg=str(e))
      break
    else:
      print(f"ClientError on entry '{i}': {e}")
      raise

  except Exception as e:
    print(f"Error on entry '{i}': {e}")
    break

  wb.save(path)

Skipping empty entry at index 17
Skipping empty entry at index 22
Error on entry '
                            Education

                                    
            University of Michigan Law School
                
JD, 2022, cum laude        
        
            Bowdoin College
                
BA, 2017, Government and Legal Studies, History        


                        ': 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 'quotaValue': '15'}]}, {'@type':